<a href="https://colab.research.google.com/github/soucs/summer-project/blob/main/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wasserstein Distance on Iris Dataset

#### Steps:
* Divide data of each class into 2 parts
* Construct multinorm distn function for each part. So total 3classes*2parts = 6 distributions. Compute pmf for each.
* Find distance between each pairts of distributions (pmfs)

In [ ]:
!pip install POT
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from ot import sinkhorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.0/790.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy import stats
from torch import tensor, softmax

from sklearn.model_selection import train_test_split
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
X = iris['data']
y = iris['target']
target_names = iris['target_names']
feature_names = iris['feature_names']

In [ ]:
target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [ ]:
X0 = X[np.where(y==0)]
X1 = X[np.where(y==1)]
X2 = X[np.where(y==2)]

X0_u, X0_v = train_test_split(X0, test_size=0.5)
X1_u, X1_v = train_test_split(X1, test_size=0.5)
X2_u, X2_v = train_test_split(X2, test_size=0.5)

In [ ]:
# Get distribution
def get_pmf(X):
  # Find mean and cov of data
  m = np.mean(X,axis=0)
  cov = np.cov(X.T)
  # Construct multinormal distribution with m and cov
  distn = stats.multivariate_normal(m,cov)
  # Calculate likelood and softmax to get pmf
  likelihood = tensor(distn.pdf(X))
  pmf = softmax(likelihood,dim=0)
  return pmf.numpy()

# Distance matrix by taking norm of differences
def get_cost_matrix(X,Y):
  cost = X[:,None] - Y[None,:]
  cost = np.linalg.norm(cost,axis=2)
  return cost

# Computing opt. coupling and wasserstein distance
# using sinkhorn algorithm
def sinkhorn_distance(X,Y,X_pmf,Y_pmf,reg=1e-1):
  cost = get_cost_matrix(X,Y)
  coupling = sinkhorn(X_pmf,Y_pmf,cost,reg)
  distance = np.sum(np.multiply(cost,coupling))
  return distance

In [ ]:
# Construct pmf for input
X0u_pmf = get_pmf(X0_u)
X0v_pmf = get_pmf(X0_v)
X1u_pmf = get_pmf(X1_u)
X1v_pmf = get_pmf(X1_v)
X2u_pmf = get_pmf(X2_u)
X2v_pmf = get_pmf(X2_v)

In [ ]:
# Calculate distance b/w diff pairs of distributions
dist0u0v = sinkhorn_distance(X0_u, X0_v, X0u_pmf, X0v_pmf)
dist0u1u = sinkhorn_distance(X0_u, X1_u, X0u_pmf, X1u_pmf)
dist0u1v = sinkhorn_distance(X0_u, X1_v, X0u_pmf, X1v_pmf)
dist0u2u = sinkhorn_distance(X0_u, X2_u, X0u_pmf, X2u_pmf)
dist0u2v = sinkhorn_distance(X0_u, X2_v, X0u_pmf, X2v_pmf)
dist1u1v = sinkhorn_distance(X1_u, X1_v, X1u_pmf, X1v_pmf)
dist1u2u = sinkhorn_distance(X1_u, X2_u, X1u_pmf, X2u_pmf)
dist1u2v = sinkhorn_distance(X1_u, X2_v, X1u_pmf, X2v_pmf)
dist2u2v = sinkhorn_distance(X2_u, X2_v, X2u_pmf, X2v_pmf)

In [ ]:
# Distance b/w distributions of same class
dist0u0v, dist1u1v, dist2u2v

(0.22106512926912397, 0.46151421132842396, 0.546816238243939)

In [ ]:
# Distance b/w 0&1 and 0&2
dist0u1u, dist0u1v, dist0u2u, dist0u2v

(3.030976488616381, 3.233723180191922, 3.2287649156292395, 2.9454820447009733)

In [ ]:
# Distance b/w 1&2
dist1u2u, dist1u2v

(0.42598641734209935, 0.19413858495364011)

* Finding the distances between distributions of class pairs:

In [ ]:
X0_pmf = get_pmf(X0)
X1_pmf = get_pmf(X1)
X2_pmf = get_pmf(X2)
dist01 = sinkhorn_distance(X0, X1, X0_pmf, X1_pmf)
dist02 = sinkhorn_distance(X0, X2, X0_pmf, X2_pmf)
dist12 = sinkhorn_distance(X1, X2, X1_pmf, X2_pmf)
dist01, dist02, dist12

(3.1259260108933793, 4.7689344259130975, 1.7804683323667352)

# On MNIST Digits Dataset

In [ ]:
!pip install POT
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from ot import sinkhorn

In [ ]:
import numpy as np
from scipy import stats
from torch import tensor, softmax

from sklearn.model_selection import train_test_split

In [ ]:
mnist_train = np.loadtxt('/content/sample_data/mnist_train_small.csv', delimiter=',') #(20000, 785)
mnist_test = np.loadtxt('/content/sample_data/mnist_test.csv', delimiter=',') #(10000, 785)

In [ ]:
X = mnist_train[:,1:]
Y = mnist_test[:,1:]

X1 = X[np.where(mnist_train[:,0]==1)]
Y1 = Y[np.where(mnist_test[:,0]==1)]

X3 = X[np.where(mnist_train[:,0]==3)]
Y3 = Y[np.where(mnist_test[:,0]==3)]

In [ ]:
# Get distribution
def get_pmf(X):
  # Find mean and cov of data
  m = np.mean(X,axis=0)
  cov = np.cov(X.T)
  # Construct multinormal distribution with m and cov
  distn = stats.multivariate_normal(m,cov, allow_singular=True)
  # Calculate likelood and softmax to get pmf
  likelihood = tensor(distn.pdf(X))
  pmf = softmax(likelihood,dim=0)
  return pmf.numpy()

# Distance matrix by taking norm of differences
def get_cost_matrix(X,Y):
  cost = X[:,None] - Y[None,:]
  cost = np.linalg.norm(cost,axis=2)
  return cost

# Computing opt. coupling and wasserstein distance
# using sinkhorn algorithm
def sinkhorn_distance(X,Y,X_pmf,Y_pmf,reg=1e-1,numItermax=10):
  cost = get_cost_matrix(X,Y)
  coupling = sinkhorn(X_pmf,Y_pmf,cost,reg)
  distance = np.sum(np.multiply(cost,coupling))
  return distance

## **Crashing! (RAM Overload)**

In [ ]:
# Construct pmf for input
X1_pmf = get_pmf(X1)
Y1_pmf = get_pmf(Y1)

X3_pmf = get_pmf(X3)
Y3_pmf = get_pmf(Y3)

# Calc distance
distX1Y1 = sinkhorn_distance(X1, Y1, X1_pmf, Y1_pmf)
distX3Y3 = sinkhorn_distance(X3, Y3, X3_pmf, Y3_pmf)

distX1X3 = sinkhorn_distance(X1, X3, X1_pmf, X3_pmf)
distX1Y3 = sinkhorn_distance(X1, Y3, X1_pmf, Y3_pmf)

distY1X3 = sinkhorn_distance(Y1, X3, Y1_pmf, X3_pmf)
distY1Y3 = sinkhorn_distance(Y1, Y3, Y1_pmf, Y3_pmf)

# Wasserstein Distance on distribution of Markov Process

In [ ]:
!pip install POT
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from ot import sinkhorn
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.0/790.0 kB 6.5 MB/s eta 0:00:00


In [ ]:
class MarkovProcess:
  def __init__(self,transition,initial,sample_space):
    self.transition = transition
    self.initial = initial
    self.sample_space = sample_space
  # Distribution at time t
  def get_distribution(self,t):
    if t<0:
      raise Exception("Sorry, no time below zero")
    else:
      return self.initial @ np.linalg.matrix_power(self.transition,t)
  # Generate a sample of process till time t
  def sample(self,t):
    if t<0:
      raise Exception("Sorry, no time below zero")
    else:
      chain = [np.random.choice(self.sample_space, p=self.get_distribution(ti)) for ti in range(t)]
      return chain
  # Generate N samples of process till time t
  def gen_samples(self,t,N):
    if t<0 or N<1:
      raise Exception("Give valid (int) inputs")
    else:
      samples = np.empty((N,t))
      for i in range(N):
        samples[i] = self.sample(t)
      return samples
  # Get prob of occurence of given sample
  def get_samp_prob(self,sample):
    p = 1
    for t,xt in enumerate(sample):
      xt_idx = np.where(self.sample_space==xt)
      pt = self.get_distribution(t)[xt_idx]
      p *= pt[0]
    return p

In [ ]:
# Fnc to get empirical distribution from samples
def get_emp_distn(samples):
  N = samples.shape[0]
  X = np.unique(samples,axis=0)
  prob = np.empty(X.shape[0])
  for i in range(X.shape[0]):
    count = (samples==X[i]).all(axis=1).sum()
    prob[i] = count/N
  return X, prob

# Fnc to get true distribution of process
def get_true_distn(mp,t):
  ss = mp.sample_space
  tiled_ss = tuple(np.tile(ss,t).reshape(t,ss.shape[0]))
  ss_t = np.array(np.meshgrid(*tiled_ss)).T.reshape(-1,t)
  prob = np.empty(ss_t.shape[0])
  for i in range(ss_t.shape[0]):
    prob[i] = mp.get_samp_prob(ss_t[i])
  return ss_t, prob

In [ ]:
samp_space = np.array([0,1])
pi_0 = np.array([0.4,0.6])
P = np.array([[0.3,0.7],[0.5,0.5]])
t = 5

# Create markov process
mp = MarkovProcess(P, pi_0, samp_space)

# Generate samples
samples = mp.gen_samples(t,100)

# Empirical distribution of samples (ED1)
X, p_emp = get_emp_distn(samples)

# True distribution (MP1)
S, p_true = get_true_distn(mp,t)

In [ ]:
# Distance matrix by taking norm of differences
def get_cost_matrix(X,Y):
  cost = X[:,None] - Y[None,:]
  cost = np.linalg.norm(cost,axis=2)
  return cost

# Computing opt. coupling and wasserstein distance
# using sinkhorn algorithm
def sinkhorn_distance(X,Y,X_pmf,Y_pmf,reg=1e-1):
  cost = get_cost_matrix(X,Y)
  coupling = sinkhorn(X_pmf,Y_pmf,cost,reg)
  distance = np.sum(np.multiply(cost,coupling))
  return distance

In [ ]:
# Distance between empirical distribution and true distribution
c = get_cost_matrix(X,S)
dist = sinkhorn_distance(X,S,p_emp,p_true)
print(f'Distance between MP1 and ED1: {dist}')

Distance between MP1 and ED1: 0.21372109819082177


In [ ]:
# Defining a second Markov Process (MP2)
samp_space = np.array([0,1])
pi_0 = np.array([0.1,0.9])
P = np.array([[0.1,0.9],[0.9,0.1]])
t = 5

# Create markov process
mp2 = MarkovProcess(P, pi_0, samp_space)
S2, S2_true = get_true_distn(mp2,t)

c2 = get_cost_matrix(S,S2)
dist2 = sinkhorn_distance(S,S2,p_true, S2_true)
dist3 = sinkhorn_distance(X,S2,p_emp, S2_true)
print(f'Distance between MP1 and MP2: {dist2}')
print(f'Distance between MP2 and ED1: {dist3}')

Distance between MP1 and MP2: 0.7823657962488051
Distance between MP2 and ED1: 0.8244211012181633
